In [1]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
cars=pd.read_excel('cars.xlsx')


Saving cars.xlsx to cars.xlsx


In [2]:
car=cars.values
data = car[:,0:4]       #(10053, 4)
target = car[: , 4]     # (10053,)


In [3]:
import numpy as np
height =  data[:,3] -  data[:,1] 
width =  data[:,2] -  data[:,0] 
s = width * height
p = 2*(width + height)

atts = np.hstack((np.ones((10053,1)) ,data, width.reshape((-1,1)) , height.reshape((-1,1)) , p.reshape((-1,1)) ,s.reshape((-1,1)) ))  #(10053, 5)

atts.shape #(bias  ,  x1 ,y1 ,x2 ,y2 , w , h , s ,p)



(10053, 9)

In [4]:
#------------------one hot encoding--------------------
bus = (target == "bus") * 1
microbus = (target == "microbus") * 1
sedan = (target == "sedan") * 1
minivan = (target == "minivan") * 1
suv = (target == "suv") * 1
truck = (target == "truck") * 1
classes = np.hstack((bus.reshape((-1,1)) , microbus.reshape((-1,1)) ,sedan.reshape((-1,1)),minivan.reshape((-1,1)),suv.reshape((-1,1)),truck.reshape((-1,1))))     #(10053, 6)
np.set_printoptions(threshold=np.inf)

classes.shape

#(bus ,  microbus  , sedan   ,  minivan   , suv   , truck)


(10053, 6)

In [5]:
#------------------shuffle-----------------------
dataset=np.hstack((atts , classes))
from sklearn.utils import shuffle
shu=shuffle(dataset)

shu.shape   #(10053, 11)

(10053, 15)

In [6]:
#---------------------test & train -----------------------------
train=shu[0:8042,:]    #(8042, 11)
test=shu[8042:,:]      #(2011, 11)
test.shape

(2011, 15)

In [7]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(train[:,:7] ,train[: ,9:15])
reg.coef_.shape                       #(6, 7)
y_tr=train[: ,9:15]
y_tr=y_tr.astype('int')
reg.score(train[:,:7], y_tr)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.3960516533606507

In [8]:
y_ts=test[: ,9:15]
y_ts=y_ts.astype('int')
reg.score(test[:,:7], y_ts)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.39454321005112586

In [12]:
u=train[:,0:7]@(reg.coef_[0,:].reshape(7,1))
an=(u > 0.25)*1
an.shape   #(8042, 1)
compare=np.hstack((u , train[:,9].reshape(8042,1) ))


tru = np.ones((8042,1))
#----------------------------------rate---------------------------------
# np.array([0,1,1,1,1,1,,1,1,1,1,,0,0,0,0,0,1,1,1,1,0,1])
# rate = (1 -  np.sum((train[:,4]-an[:,0])**2)/8042)
# f11 = np.sum(((train[:,4] == an[:,0]) and ( train[:,4] == tru ) )*1)
# rate=f11/(f)
from scipy.spatial import distance
1 - distance.jaccard(train[:,9] , an[:,0])
# compare

# pred1 = reg.predict(train[:,:7])

# # rate = (1 -  np.sum((y1-v11)**2)/150)
# # compare
# # rate
# pred1.shape   #(8042, 6)

0.357429718875502

logestic regression



In [10]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
 
 
y_tr=train[: ,9]
y_tr=y_tr.astype('int')
 
 
clf.fit(train[:,:7] ,y_tr )
 
# clf.predict(train[:,:7])
 
# clf.predict_proba(X[:2, :])
clf.score(train[:,:7], y_tr)     


0.9440437702064163

In [11]:
y_ts=test[: ,9]
y_ts=y_ts.astype('int')
 
clf.score(test[:,:7], y_ts)  


0.939830929885629

In [19]:

tru = np.ones((8042,1))

from scipy.spatial import distance
1 - distance.jaccard(train[:,9] , clf.predict(train[:,:7]).reshape(-1,1))

#compare=np.hstack((train[:,9].reshape((-1,1)),clf.predict(train[:,:7]).reshape(-1,1)))
#compare


0.19499105545617168

decision tree

In [20]:
>>> from sklearn import tree
X = [[0, 0], [1, 1]]
Y = [0, 1]
clft = tree.DecisionTreeClassifier()
clft = clft.fit(train[:,:7],  y_tr)
clft

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
tree.plot_tree(clft) 

In [ ]:
clft.score(train[:,:7], y_tr)

1.0

In [ ]:
clft.score(test[:,:7], y_ts)

0.9621890547263682

In [ ]:
pred_lrt=clft.predict(train[:,:7])
1 - distance.jaccard(train[:,9] , pred_lrt)

1.0

In [ ]:
pred_tst=clft.predict(test[:,:7])
1 - distance.jaccard(test[:,9] , pred_tst)

0.4722222222222222

multi level Perceptron

In [27]:
from sklearn.neural_network import MLPClassifier
clfnn = MLPClassifier(solver='lbfgs', alpha=0.1, hidden_layer_sizes=(4, 2), random_state=1)
clfnn.fit(train[:,:7],  y_tr)

MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(4, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [28]:
clfnn.score(train[:,:7], y_tr)

0.9444168117383736

In [29]:
clfnn.score(test[:,:7], y_ts)

0.9448035803083044

In [35]:
compare=np.hstack((train[:,9].reshape((-1,1)),clfnn.predict(train[:,:7]).reshape(-1,1)))
compare


array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0,